# Preprocessing Wellington Data

## Goals

- Split running time into, hourly run time intervals
- Calculate Average speed of a bus (distance/time)
- Calculate Average Occupancy (passenger Km/ actual km
- Calculate the per person carbon emissions reduction

## Further Goals

- Create Heat Map
- Learn how to upload data to mongoDB
- Integrate heat map into Bean

## Loading Data

In [21]:
import pandas as pd

jan = pd.read_csv("Wellington Raw Daily Data/Jan 1-3 2022.csv")
jan

,Date,Route,Route Variant,Direction,Trip Number,Actual Vehicle Type,Vehicle Number,Vehicle Emissions Standard,Start Minute (Sched),Actual Running Time,Actual In-Service KM,Passenger km,Scheduled In-Service km,Cancelled Trips,Sched Running Time per Trip
0,01-Jan-2022,1,1_1,Inbound,4000.0,ELVDD,3705,ELECTRIC,06:45,931,21.239,120.695,21.239,0,"3,720"
1,01-Jan-2022,1,1_1,Inbound,4060.0,ELVDD,3704,ELECTRIC,07:45,"3,134",21.239,42.381,21.239,0,"3,720"
2,01-Jan-2022,1,1_1,Inbound,4100.0,LV,3412,EURO6,08:15,"3,094",21.239,127.285,21.239,0,"3,720"
3,01-Jan-2022,1,1_1,Inbound,4140.0,LV,3401,EURO6,08:45,"3,212",21.239,69.496,21.239,0,"4,080"
4,01-Jan-2022,1,1_1,Inbound,4180.0,ELVDD,3705,ELECTRIC,09:15,948,21.239,73.054,21.239,0,"4,080"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5941,03-Jan-2022,WRL,WRL_1,Inbound,1613.0,Unknown,4398,Unknown,07:45,"2,237",90.803,0.000,90.803,0,"6,000"
5942,03-Jan-2022,WRL,WRL_1,Inbound,1615.0,Unknown,4398,Unknown,16:45,"6,107",90.803,0.000,90.803,0,"6,000"
5943,03-Jan-2022,WRL,WRL_2,Outbound,1614.0,Unknown,4398,Unknown,09:55,"2,400",90.805,0.000,90.805,0,"6,000"
5944,03-Jan-2022,WRL,WRL_2,Outbound,1616.0,Unknown,4398,Unknown,18:55,"3,494",90.805,0.000,90.805,0,"6,000"


#### The bottom row of each data frame must be removed due to it being a total column.

In [19]:
jan = jan.drop(jan.tail(1).index)

#### Now we must bind multiple data frames together so that we can have a month of data

In [24]:
jan2 = pd.read_csv("Wellington Raw Daily Data/Jan 4-6 2022.csv")
jan2

,Date,Route,Route Variant,Direction,Trip Number,Actual Vehicle Type,Vehicle Number,Vehicle Emissions Standard,Start Minute (Sched),Actual Running Time,Actual In-Service KM,Passenger km,Scheduled In-Service km,Cancelled Trips,Sched Running Time per Trip
0,04-Jan-2022,1,1_1,Inbound,7000.0,ELVDD,3705,ELECTRIC,07:45,"3,123",21.239,152.205,21.239,0,"3,480"
1,04-Jan-2022,1,1_1,Inbound,7040.0,DD,3522,EURO6,08:15,"3,212",21.239,34.119,21.239,0,"3,480"
2,04-Jan-2022,1,1_1,Inbound,7080.0,LV,3411,EURO6,08:45,720,21.239,64.045,21.239,0,"3,480"
3,04-Jan-2022,1,1_1,Inbound,7120.0,LV,3432,EURO6,09:15,"3,455",21.239,301.589,21.239,0,"3,660"
4,04-Jan-2022,1,1_1,Inbound,7160.0,ELVDD,3703,ELECTRIC,09:45,"3,326",21.239,71.795,21.239,0,"3,840"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9373,06-Jan-2022,WRL,WRL_1,Inbound,1615.0,Unknown,4064,Unknown,16:45,"5,508",90.803,0.000,90.803,0,"6,000"
9374,06-Jan-2022,WRL,WRL_2,Outbound,1608.0,Unknown,4063,Unknown,17:30,"4,589",90.805,0.000,90.805,0,"6,300"
9375,06-Jan-2022,WRL,WRL_2,Outbound,1614.0,Unknown,4064,Unknown,09:55,"6,434",90.805,0.000,90.805,0,"6,000"
9376,06-Jan-2022,WRL,WRL_2,Outbound,1616.0,Unknown,4064,Unknown,18:55,"2,209",90.805,0.000,90.805,0,"6,000"


In [25]:
jan3 = pd.read_csv("Wellington Raw Daily Data/Jan 7-9 2022.csv")
jan3

,Date,Route,Route Variant,Direction,Trip Number,Actual Vehicle Type,Vehicle Number,Vehicle Emissions Standard,Start Minute (Sched),Actual Running Time,Actual In-Service KM,Passenger km,Scheduled In-Service km,Cancelled Trips,Sched Running Time per Trip
0,07-Jan-2022,1,1_1,Inbound,1020.0,Unknown,3842,Unknown,23:45,877,21.239,26.315,21.239,0,"3,120"
1,07-Jan-2022,1,1_1,Inbound,1040.0,ELVDD,3707,ELECTRIC,05:45,"2,353",21.239,75.832,21.239,0,"3,480"
2,07-Jan-2022,1,1_1,Inbound,1120.0,LV,3445,EURO6,06:45,"3,404",21.239,269.753,21.239,0,"4,020"
3,07-Jan-2022,1,1_1,Inbound,1180.0,LV,3429,EURO6,07:05,"4,444",21.239,138.245,21.239,0,"4,860"
4,07-Jan-2022,1,1_1,Inbound,1240.0,LV,3422,EURO6,07:25,"3,105",21.239,243.870,21.239,0,"5,580"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8089,09-Jan-2022,WRL,WRL_1,Inbound,1613.0,Unknown,4398,Unknown,07:45,"3,174",90.803,0.000,90.803,0,"6,000"
8090,09-Jan-2022,WRL,WRL_1,Inbound,1615.0,Unknown,4398,Unknown,16:45,"3,357",90.803,0.000,90.803,0,"6,000"
8091,09-Jan-2022,WRL,WRL_2,Outbound,1614.0,Unknown,4398,Unknown,09:55,"4,536",90.805,0.000,90.805,0,"6,000"
8092,09-Jan-2022,WRL,WRL_2,Outbound,1616.0,Unknown,4398,Unknown,18:55,"4,132",90.805,0.000,90.805,0,"6,000"
